In [1]:
import requests
import logging

# Logging setup
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s]: %(message)s")


class ImpactCO2Model:
    def __init__(self, api_key):
        """
        Initialize the ImpactCO2Model with API key.
        :param api_key: Your API key for ImpactCO2.
        """
        self.api_base_url = "https://impactco2.fr/api/v1"
        self.api_key = api_key
        self.headers = {"Authorization": f"Bearer {self.api_key}"}

    def _make_request(self, endpoint, params=None):
        """
        Make a GET request to the ImpactCO2 API.
        :param endpoint: API endpoint (e.g., 'heating', 'transport').
        :param params: Query parameters for the API request.
        :return: Parsed JSON response or None on failure.
        """
        url = f"{self.api_base_url}/{endpoint}"
        try:
            response = requests.get(url, headers=self.headers, params=params, timeout=5)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            logging.error(f"Error while querying ImpactCO2 API: {e}")
            return None

    def get_heating_emissions(self, heating_type, energy_consumption):
        """
        Get CO2 emissions for a specific heating type.
        :param heating_type: Type of heating (e.g., 'electricity', 'gas', 'fuel').
        :param energy_consumption: Energy consumption in kWh.
        :return: CO2 emissions in kg or None.
        """
        params = {"type": heating_type, "consumption": energy_consumption}
        data = self._make_request("heating", params)
        if data:
            emissions = data.get("co2_emissions")
            logging.info(f"Heating Type: {heating_type}, Consumption: {energy_consumption} kWh, Emissions: {emissions} kg CO₂e")
            return emissions
        return None

    def get_transport_emissions(self, transport_mode, distance):
        """
        Get CO2 emissions for a specific transport mode.
        :param transport_mode: Mode of transport (e.g., 'car', 'bus', 'train', 'plane').
        :param distance: Distance traveled in km.
        :return: CO2 emissions in kg or None.
        """
        params = {"mode": transport_mode, "distance": distance}
        data = self._make_request("transport", params)
        if data:
            emissions = data.get("co2_emissions")
            logging.info(f"Transport Mode: {transport_mode}, Distance: {distance} km, Emissions: {emissions} kg CO₂e")
            return emissions
        return None


In [1]:
import requests
import logging

# Logging setup
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s]: %(message)s")

# Transport options dictionary
TRANSPORT_OPTIONS = {
    1: "Avion",
    2: "TGV",
    3: "Intercités",
    4: "Voiture thermique",
    5: "Voiture électrique",
    6: "Autocar thermique",
    7: "Vélo",
    8: "Vélo à assistance électrique",
    9: "Bus thermique",
    10: "Tramway",
    11: "Métro",
    12: "Scooter ou moto légère thermique",
    13: "Moto thermique",
    14: "RER ou Transilien",
    15: "TER",
    16: "Bus électrique",
    17: "Trottinette à assistance électrique",
    21: "Bus (GNV)",
    22: "Covoiturage thermique (1 passager)",
    23: "Covoiturage thermique (2 passagers)",
    24: "Covoiturage thermique (3 passagers)",
    25: "Covoiturage thermique (4 passagers)",
    26: "Covoiturage électrique (1 passager)",
    27: "Covoiturage électrique (2 passagers)",
    28: "Covoiturage électrique (3 passagers)",
    29: "Covoiturage électrique (4 passagers)",
    30: "Marche",
}

# Heating options dictionary
HEATING_OPTIONS = {
    1: "Chauffage au gaz",
    2: "Chauffage au fioul",
    3: "Chauffage électrique",
    4: "Chauffage avec une pompe à chaleur",
    5: "Chauffage avec un poêle à granulés",
    6: "Chauffage avec un poêle à bois",
    7: "Chauffage via un réseau de chaleur",
}


def display_options(options_dict, title):
    """Display options from a dictionary."""
    print(f"\n{title}:")
    for key, value in options_dict.items():
        print(f"{key}: {value}")


def get_transport_emissions(km, transport_id, display_all=1, ignore_radiative_forcing=1, occupancy_rate=1, include_construction=1, language="fr"):
    """
    Get CO2 emissions for transport.
    """
    url = "https://impactco2.fr/api/v1/transport"
    params = {
        "km": km,
        "displayAll": display_all,
        "transports": transport_id,
        "ignoreRadiativeForcing": ignore_radiative_forcing,
        "occupencyRate": occupancy_rate,
        "includeConstruction": include_construction,
        "language": language,
    }

    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            if "data" in data:
                return data["data"]
            else:
                logging.warning("No emissions data available for the transport query.")
        else:
            logging.error(f"Failed to fetch transport data. HTTP Status: {response.status_code}")
    except Exception as e:
        logging.error(f"An error occurred while fetching transport data: {e}")
    return None


def suggest_alternative_transports(km, current_transport_id):
    """
    Suggest alternative transport modes with lower CO2 emissions.
    """
    # Get emissions for all transport modes
    all_emissions = get_transport_emissions(km, ",".join(map(str, TRANSPORT_OPTIONS.keys())))
    if not all_emissions:
        logging.warning("Could not retrieve emissions data to suggest alternatives.")
        return None

    # Sort by CO2 emissions
    all_emissions_sorted = sorted(all_emissions, key=lambda x: x["value"])

    # Find the alternatives with lower emissions than the current transport
    current_emissions = next((x for x in all_emissions if x["id"] == current_transport_id), None)
    if not current_emissions:
        logging.warning(f"Could not find emissions data for transport ID {current_transport_id}.")
        return None

    alternatives = [item for item in all_emissions_sorted if item["value"] < current_emissions["value"]]
    return alternatives[:3]  # Return top 3 alternatives


def get_heating_emissions(m2, heating_id, language="fr"):
    """
    Get CO2 emissions for heating.
    """
    url = "https://impactco2.fr/api/v1/chauffage"
    params = {
        "m2": m2,
        "chauffages": heating_id,
        "language": language,
    }

    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            if "data" in data:
                return data["data"]
            else:
                logging.warning("No emissions data available for the heating query.")
        else:
            logging.error(f"Failed to fetch heating data. HTTP Status: {response.status_code}")
    except Exception as e:
        logging.error(f"An error occurred while fetching heating data: {e}")
    return None


def main():
    """
    Main function to handle user input and fetch emissions data.
    """
    logging.info("Welcome to the CO2 emissions calculator!")

    while True:
        logging.info("Choose an option:")
        logging.info("1. Calculate emissions for transport")
        logging.info("2. Calculate emissions for heating")
        logging.info("3. Exit")

        choice = input("Enter your choice (1/2/3): ").strip()

        if choice == "1":
            # Transport emissions
            display_options(TRANSPORT_OPTIONS, "Transport Options")
            try:
                km = float(input("Enter the distance in kilometers: ").strip())
                transport_id = int(input("Enter the transport type ID: ").strip())
                if transport_id not in TRANSPORT_OPTIONS:
                    logging.error("Invalid transport ID. Please select from the list.")
                    continue
                transport_data = get_transport_emissions(km, transport_id)
                if transport_data:
                    print(f"\nTransport emissions data for {km} km using {TRANSPORT_OPTIONS[transport_id]}:")
                    for item in transport_data:
                        print(f"- {item['name']}: {item['value1']} kg CO2e")

                    # Suggest alternatives
                    alternatives = suggest_alternative_transports(km, transport_id)
                    if alternatives:
                        print("\nSuggested alternative transport options with lower CO2 emissions:")
                        for alt in alternatives:
                            print(f"- {alt['name']}: {alt['value']} kg CO2e")
            except ValueError:
                logging.error("Invalid input for kilometers or transport ID. Please enter valid numbers.")
        elif choice == "2":
            # Heating emissions
            display_options(HEATING_OPTIONS, "Heating Options")
            try:
                m2 = float(input("Enter the surface area in square meters: ").strip())
                heating_id = int(input("Enter the heating type ID: ").strip())
                if heating_id not in HEATING_OPTIONS:
                    logging.error("Invalid heating ID. Please select from the list.")
                    continue
                heating_data = get_heating_emissions(m2, heating_id)
                if heating_data:
                    print(f"\nHeating emissions data for {m2} m² using {HEATING_OPTIONS[heating_id]}:")
                    for item in heating_data:
                        print(f"- {item['name']}: {item['ecv']} kg CO2e")
            except ValueError:
                logging.error("Invalid input for surface area or heating ID. Please enter valid numbers.")
        elif choice == "3":
            logging.info("Exiting the program. Goodbye!")
            break
        else:
            logging.error("Invalid choice. Please select 1, 2, or 3.")


if __name__ == "__main__":
    main()


2025-01-20 20:48:42,985 [INFO]: Welcome to the CO2 emissions calculator!
2025-01-20 20:48:42,985 [INFO]: Choose an option:
2025-01-20 20:48:42,985 [INFO]: 1. Calculate emissions for transport
2025-01-20 20:48:42,985 [INFO]: 2. Calculate emissions for heating
2025-01-20 20:48:42,985 [INFO]: 3. Exit
2025-01-20 20:48:42,985 [ERROR]: Invalid choice. Please select 1, 2, or 3.
2025-01-20 20:48:42,985 [INFO]: Choose an option:
2025-01-20 20:48:42,985 [INFO]: 1. Calculate emissions for transport
2025-01-20 20:48:42,995 [INFO]: 2. Calculate emissions for heating
2025-01-20 20:48:42,995 [INFO]: 3. Exit
2025-01-20 20:48:45,834 [INFO]: Exiting the program. Goodbye!


In [ ]:
import requests
import logging

# Logging setup
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s]: %(message)s")

# Transport options dictionary
TRANSPORT_OPTIONS = {
    1: "Avion",
    2: "TGV",
    3: "Intercités",
    4: "Voiture thermique",
    5: "Voiture électrique",
    6: "Autocar thermique",
    7: "Vélo",
    8: "Vélo à assistance électrique",
    9: "Bus thermique",
    10: "Tramway",
    11: "Métro",
    12: "Scooter ou moto légère thermique",
    13: "Moto thermique",
    14: "RER ou Transilien",
    15: "TER",
    16: "Bus électrique",
    17: "Trottinette à assistance électrique",
    21: "Bus (GNV)",
    22: "Covoiturage thermique (1 passager)",
    23: "Covoiturage thermique (2 passagers)",
    24: "Covoiturage thermique (3 passagers)",
    25: "Covoiturage thermique (4 passagers)",
    26: "Covoiturage électrique (1 passager)",
    27: "Covoiturage électrique (2 passagers)",
    28: "Covoiturage électrique (3 passagers)",
    29: "Covoiturage électrique (4 passagers)",
    30: "Marche",
}

def get_emissions_for_all_transports(km=1, language="fr"):
    """
    Fetch emissions for all transport options from the Impact CO2 API.
    
    Parameters:
    - km: Distance in kilometers for which emissions are calculated (default: 1 km).
    - language: Language for the API response (default: "fr").
    
    Returns:
    - A dictionary containing emissions data for each transport type.
    """
    url = "https://impactco2.fr/api/v1/transport"
    emissions_data = {}

    # Iterate over all transport IDs and fetch emissions data
    for transport_id, transport_name in TRANSPORT_OPTIONS.items():
        params = {
            "km": km,
            "displayAll": 1,
            "transports": transport_id,
            "ignoreRadiativeForcing": 1,
            "occupencyRate": 1,
            "includeConstruction": 1,
            "language": language,
        }

        try:
            response = requests.get(url, params=params)
            if response.status_code == 200:
                data = response.json()
                if "data" in data:
                    # Extract emission data for the transport
                    emission_value = data["data"][0]["value"] if data["data"] else None
                    emissions_data[transport_name] = emission_value
                    logging.info(f"Fetched emissions for {transport_name}: {emission_value} kg CO2/km")
                else:
                    logging.warning(f"No emissions data for {transport_name}.")
            else:
                logging.error(f"Failed to fetch data for {transport_name}. HTTP Status: {response.status_code}")
        except Exception as e:
            logging.error(f"An error occurred while fetching data for {transport_name}: {e}")
    
    return emissions_data

def main():
    """
    Main function to fetch and display emissions for all transport types.
    """
    logging.info("Fetching emissions data for all transport types...")
    emissions_data = get_emissions_for_all_transports(km=1)  # Default distance: 1 km

    if emissions_data:
        print("\nEmissions data (kg CO2/km) for all transport types:")
        for transport, emission in emissions_data.items():
            print(f"- {transport}: {emission} kg CO2/km")
    else:
        logging.warning("No emissions data was retrieved.")

if __name__ == "__main__":
    main()
